In [410]:
# first level direction: alwasy left
# generate clock wise
def xy2d(n, x, y):
    d = 0
    s = int(n / 2)
    while s > 0:
        rx = (x & s) > 0
        ry = (y & s) > 0
        if rx == 0 and ry == 0:
            d += 0
        if rx == 1 and ry == 0:
            d += s*s
        if rx == 1 and ry == 1:
            d += s*s*2
        if rx == 0 and ry == 1:
            d += s*s*3   
#         d += s * s * ((3 * rx) ^ ry)
        x,y = rot(s, x, y, rx, ry)
        s = int(s/2)
    return d

# def d2xy(n, d):
#     t = d
#     x = 0
#     y = 0
#     s = 1
#     while s < n:
#         rx = 1 & int(t/2)
#         ry = 1 & (t^rx)
#         x,y = rot(s, x, y, rx, ry)
#         x += s * rx
#         y += s * ry
#         t = int(t/4)
#         s *= 2
#     return x, y

def rot(n, x, y, rx, ry):
    if rx == 0 : # the upper level direction remains the same, so only change the lower level
        if ry == 1 :
            x = n-1 - x
            y = n-1 - y
        t  = x
        x = y
        y = t
    return x, y

# ===== Testing =====
# for i in range(8):
#     for j in range(8):
#         print(xy2d(8, i, j))

In [411]:
# import ipdb
# ipdb.set_trace()
# print(xy2d(1<<14, 2506754>>8, 3194880>>8))
# print(xy2d(1<<14, 2818050>>8, 1785859>>8))
# print(xy2d(1<<14, 2506756>>8, 3194880>>8))

In [412]:
from enum import Enum
class Direction(Enum):
    UP = 0
    LEFT = 1
    RIGHT = 2
    DOWN = 3

class ClockDirection(Enum):
    CLOCKWISE = 0
    COUNTERCLOCKWISE = 1
    UNAVAILABLE = 2

class Quadrant(Enum):
    LOWERLEFT = 0
    LOWERRIGHT = 1
    UPPERRIGHT = 2
    UPPERLEFT = 3
    
next_level_direction=[[Direction.UP, Direction.UP, Direction.RIGHT, Direction.LEFT],
                      [Direction.DOWN, Direction.LEFT, Direction.LEFT, Direction.UP],
                      [Direction.RIGHT, Direction.DOWN, Direction.UP, Direction.RIGHT],
                      [Direction.LEFT, Direction.RIGHT, Direction.DOWN, Direction.DOWN]]

next_level_clock_direction = [[ClockDirection.CLOCKWISE,ClockDirection.COUNTERCLOCKWISE,ClockDirection.COUNTERCLOCKWISE,ClockDirection.UNAVAILABLE],
                           [ClockDirection.CLOCKWISE,ClockDirection.COUNTERCLOCKWISE,ClockDirection.UNAVAILABLE,ClockDirection.CLOCKWISE],
                           [ClockDirection.CLOCKWISE,ClockDirection.UNAVAILABLE,ClockDirection.COUNTERCLOCKWISE,ClockDirection.CLOCKWISE],
                           [ClockDirection.UNAVAILABLE,ClockDirection.COUNTERCLOCKWISE,ClockDirection.COUNTERCLOCKWISE,ClockDirection.CLOCKWISE]]

# ===== Testing =====
# for i in range(4):
#     for j in range(4):
#         print(next_level_direction[i][j])
# vard = Direction.UP
# if vard.name == "UP":
#     print("yes") # yes
# print(vard.name)
# print(vard.value)
# if vard == Direction.UP:
#     print("yes")
# for dir in Direction:
#     print(dir)

In [413]:
# dictionary for interval:
# lower_x
# upper_X
# lower_y
# upper_y
# lower_hilbert
# upper_hilbert


def range_decompose(max_bits, current_bits, down_x, up_x, down_y, up_y, center_x, center_y, pre_dir, this_dir, intervals):
    
    # if there is only 1 element here
    if current_bits == 0:
#         print("reach here!!!!!!!!")
        hvalue = xy2d(1<<max_bits, down_x, down_y)
        interval = {"lower_x":down_x, "upper_x":up_x, "lower_y":down_y,"upper_y":up_y,"lower_hilbert":hvalue,"upper_hilbert":hvalue}
        intervals.append(interval);
        return
    
    half_region = 1 << (current_bits - 1)
    clock_dir = next_level_clock_direction[pre_dir.value][this_dir.value]
    if current_bits == max_bits:
        clocl_dir = ClockDirection.COUNTERCLOCKWISE
        
    # check if the range contains the entire  region
    if down_x == center_x - half_region and down_y == center_y - half_region and up_x == center_x + half_region-1 and up_y == center_y + half_region-1:
        # if yes, return this region as an interval
        lower_x = 0
        upper_x = 0
        lower_y = 0
        upper_y = 0
        if clock_dir.name == "CLOCKWISE":
            if this_dir.name == "UP":
                lower_x = up_x
                upper_x = down_x
                lower_y = up_y
                upper_y = up_y
            elif this_dir.name == "LEFT":
                lower_x = down_x
                upper_x = down_x
                lower_y = up_y
                upper_y = down_y
            elif this_dir.name == "RIGHT":
                lower_x = up_x
                upper_x = up_x
                lower_y = down_y
                upper_y = up_y
            elif this_dir.name == "DOWN":
                lower_x = down_x
                upper_x = up_x
                lower_y = down_y
                upper_y = down_y
        elif clock_dir.name == "COUNTERCLOCKWISE":
            if this_dir.name == "UP":
                lower_x = down_x
                upper_x = up_x
                lower_y = up_y
                upper_y = up_y
            elif this_dir.name == "LEFT":
                lower_x = down_x
                upper_x = down_x
                lower_y = down_y
                upper_y = up_y
            elif this_dir.name == "RIGHT":
                lower_x = up_x
                upper_x = up_x
                lower_y = up_y
                upper_y = down_y
            elif this_dir.name == "DOWN":
                lower_x = up_x
                upper_x = down_x
                lower_y = down_y
                upper_y = down_y
                
        lowhvalue = xy2d(1<<max_bits,lower_x,lower_y)
        uphvalue = xy2d(1<<max_bits,upper_x,upper_y)
        interval = {"lower_x":lower_x, "upper_x":upper_x, "lower_y":lower_y,"upper_y":upper_y,"lower_hilbert":lowhvalue,"upper_hilbert":uphvalue}
        intervals.append(interval)
    else:
        # if not, further decompose it
        current_bits -= 1
        if current_bits <= 0:
            half_region = 0
        else:
            half_region = 1 << (current_bits - 1)
        
        if clock_dir.name == "CLOCKWISE":
            if this_dir.name == "UP":
                parent_dir = Direction.UP
                if up_x >= center_x and up_y >= center_y: # upper right
                    quad = Quadrant.UPPERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, max(center_y,down_y), up_y, center_x+half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and down_y < center_y: # lower right
                    quad = Quadrant.LOWERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, down_y, min(center_y-1,up_y), center_x+half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and down_y < center_y: # lower left\
                    quad = Quadrant.LOWERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), down_y, min(center_y-1,up_y), center_x-half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and up_y >= center_y: # upper left
                    quad = Quadrant.UPPERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), max(center_y,down_y), up_y, center_x-half_region, center_y+half_region, parent_dir, child_dir, intervals)
            elif this_dir.name == "LEFT":
                parent_dir = Direction.LEFT
                if down_x < center_x and up_y >= center_y: # upper left
                    quad = Quadrant.UPPERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), max(center_y,down_y), up_y, center_x-half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and up_y >= center_y: # upper right
                    quad = Quadrant.UPPERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, max(center_y,down_y), up_y, center_x+half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and down_y < center_y: # lower right
                    quad = Quadrant.LOWERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, down_y, min(center_y-1,up_y), center_x+half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and down_y < center_y: # lower left\
                    quad = Quadrant.LOWERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), down_y, min(center_y-1,up_y), center_x-half_region, center_y-half_region, parent_dir, child_dir, intervals)
            elif this_dir.name == "RIGHT":
                parent_dir = Direction.RIGHT
                if up_x >= center_x and down_y < center_y: # lower right
                    quad = Quadrant.LOWERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, down_y, min(center_y-1,up_y), center_x+half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and down_y < center_y: # lower left\
                    quad = Quadrant.LOWERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), down_y, min(center_y-1,up_y), center_x-half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and up_y >= center_y: # upper left
                    quad = Quadrant.UPPERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), max(center_y,down_y), up_y, center_x-half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and up_y >= center_y: # upper right
                    quad = Quadrant.UPPERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, max(center_y,down_y), up_y, center_x+half_region, center_y+half_region, parent_dir, child_dir, intervals)
            elif this_dir.name == "DOWN":
                parent_dir = Direction.DOWN
                if down_x < center_x and down_y < center_y: # lower left\
                    quad = Quadrant.LOWERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), down_y, min(center_y-1,up_y), center_x-half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and up_y >= center_y: # upper left
                    quad = Quadrant.UPPERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), max(center_y,down_y), up_y, center_x-half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and up_y >= center_y: # upper right
                    quad = Quadrant.UPPERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, max(center_y,down_y), up_y, center_x+half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and down_y < center_y: # lower right
                    quad = Quadrant.LOWERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, down_y, min(center_y-1,up_y), center_x+half_region, center_y-half_region, parent_dir, child_dir, intervals)
        elif clock_dir.name == "COUNTERCLOCKWISE":
            if this_dir.name == "UP":
                parent_dir = Direction.UP
                if down_x < center_x and up_y >= center_y: # upper left
                    quad = Quadrant.UPPERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), max(center_y,down_y), up_y, center_x-half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and down_y < center_y: # lower left\
                    quad = Quadrant.LOWERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), down_y, min(center_y-1,up_y), center_x-half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and down_y < center_y: # lower right
                    quad = Quadrant.LOWERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, down_y, min(center_y-1,up_y), center_x+half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and up_y >= center_y: # upper right
                    quad = Quadrant.UPPERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, max(center_y,down_y), up_y, center_x+half_region, center_y+half_region, parent_dir, child_dir, intervals)
            elif this_dir.name == "LEFT":
                parent_dir = Direction.LEFT
                if down_x < center_x and down_y < center_y: # lower left
                    quad = Quadrant.LOWERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), down_y, min(center_y-1,up_y), center_x-half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and down_y < center_y: # lower right
                    quad = Quadrant.LOWERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, down_y, min(center_y-1,up_y), center_x+half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and up_y >= center_y: # upper right
                    quad = Quadrant.UPPERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, max(center_y,down_y), up_y, center_x+half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and up_y >= center_y: # upper left
                    quad = Quadrant.UPPERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), max(center_y,down_y), up_y, center_x-half_region, center_y+half_region, parent_dir, child_dir, intervals)
            elif this_dir.name == "RIGHT":
                parent_dir = Direction.RIGHT
                if up_x >= center_x and up_y >= center_y: # upper right
                    quad = Quadrant.UPPERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, max(center_y,down_y), up_y, center_x+half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and up_y >= center_y: # upper left
                    quad = Quadrant.UPPERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), max(center_y,down_y), up_y, center_x-half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and down_y < center_y: # lower left
                    quad = Quadrant.LOWERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), down_y, min(center_y-1,up_y), center_x-half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and down_y < center_y: # lower right
                    quad = Quadrant.LOWERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, down_y, min(center_y-1,up_y), center_x+half_region, center_y-half_region, parent_dir, child_dir, intervals)
            elif this_dir.name == "DOWN":
                parent_dir = Direction.DOWN
                if up_x >= center_x and down_y < center_y: # lower right
                    quad = Quadrant.LOWERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, down_y, min(center_y-1,up_y), center_x+half_region, center_y-half_region, parent_dir, child_dir, intervals)
                if up_x >= center_x and up_y >= center_y: # upper right
                    quad = Quadrant.UPPERRIGHT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, max(center_x,down_x), up_x, max(center_y,down_y), up_y, center_x+half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and up_y >= center_y: # upper left
                    quad = Quadrant.UPPERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), max(center_y,down_y), up_y, center_x-half_region, center_y+half_region, parent_dir, child_dir, intervals)
                if down_x < center_x and down_y < center_y: # lower left
                    quad = Quadrant.LOWERLEFT
                    child_dir = next_level_direction[parent_dir.value][quad.value]
                    range_decompose(max_bits, current_bits, down_x, min(center_x-1,up_x), down_y, min(center_y-1,up_y), center_x-half_region, center_y-half_region, parent_dir, child_dir, intervals)

# ===== Testing =====
# intervals = []
# range_decompose(3,3,2,4,2,4,4,4,Direction.LEFT,Direction.LEFT,intervals)
# range_decompose(3,0,2,4,2,4,4,4,Direction.LEFT,Direction.LEFT,intervals)
# range_decompose(3,1,2,3,2,3,3,3,Direction.LEFT,Direction.LEFT,intervals)
# range_decompose(3,1,2,3,2,3,3,3,Direction.DOWN,Direction.DOWN,intervals)
# print(intervals)

# import ipdb
# ipdb.set_trace()
# intervals = []
# # range_decompose(5,5,8,15,8,23,16,16,Direction.LEFT,Direction.LEFT,intervals)
# range_decompose(6,6,32,34,32,43,32,32,Direction.LEFT,Direction.LEFT,intervals)
# print(intervals)
# print(len(intervals))
# intervals = intervals[0]
# print(intervals[0])
# print(intervals[0]["lower_x"])
# for inter in intervals:
#     print(intervals)

In [414]:
def merge_intervals(intervals):
    if len(intervals) <= 1:
        return intervals
    merged_intervals = []
    i = 0
    j = 0
    while i < len(intervals):
        temp_upper = intervals[i]["upper_hilbert"]
        j = 1
        while i+j < len(intervals) and intervals[i+j]["lower_hilbert"] == temp_upper + 1:
            temp_upper = intervals[i+j]["upper_hilbert"]
            j += 1
        interval = {"lower_x":intervals[i]["lower_x"], "upper_x":intervals[i+j-1]["upper_x"], 
                    "lower_y":intervals[i]["lower_y"],"upper_y":intervals[i+j-1]["upper_y"],
                    "lower_hilbert":intervals[i]["lower_hilbert"],"upper_hilbert":intervals[i+j-1]["upper_hilbert"]}
        merged_intervals.append(interval)
        i += j
    return merged_intervals

def get_intervals(lower_x, upper_x, lower_y, upper_y, max_bits = 22):
    intervals = []
    range_decompose(max_bits,max_bits,lower_x,upper_x,lower_y,upper_y,1<<(max_bits-1),1<<(max_bits-1),Direction.LEFT,Direction.LEFT,intervals)
    intervals = merge_intervals(intervals)
    return intervals
    
# ===== Testing =====
# import ipdb
# ipdb.set_trace()
#print(get_intervals(2,4,2,4))
# print(get_intervals(8,15,8,23,5))
# print(get_intervals(2097152,4194303,2097152,4194303,22))
# print(get_intervals(32,34,32,43,6))
# print(get_intervals(2,4,2,4,4))
# print(get_intervals(10,13,11,12,5))

In [415]:
# ===== Testing =====
# import ipdb
# ipdb.set_trace()
# intervals = []
# range_decompose(3,3,2,4,2,4,4,4,Direction.LEFT,Direction.LEFT,intervals)
# print(intervals)
# intervals = merge_intervals(intervals)
# print(intervals)

In [416]:
# print(xy2d(4194306, 1600000, 1900000))

In [417]:
import pandas as pd
df = pd.read_csv("./Data/HilbertSortedPOIs.csv")
print(df)
TOTAL = len(df)+1
COL = df.columns

# import numpy as np
# print(np.any(np.isnan(df))) # should be false
# print(np.all(np.isfinite(df))) # should be true
# from sklearn import preprocessing
# scaler = preprocessing.StandardScaler()
# print(scaler.fit(df.iloc[:,0:5]))
# print(scaler.mean_)
# df.iloc[:,0:5]=scaler.transform(df.iloc[:,0:5])
# print(df)

# ===== Testing =====
# df1 = pd.DataFrame(data=[[-76.3099,22.4995,273802,2024995,17344470968016]], columns=["x","y","x_","y_","hilbert"],index=[0], dtype=object)
# df1 = pd.DataFrame(data=[[-10,10,1600000,1900000,2819270465194]], columns=["x","y","x_","y_","hilbert"],index=[0], dtype=object)
# print(df1)
# df1.iloc[0,:] = scaler.transform(df1.iloc[0,:].reshape(1,-1))
# print(df1)
# df[df.duplicated('hilbert')]

               x         y   x_   y_  hilbert    order  prediction
0       -83.8626 -160.3120   29   48     3323        0         NaN
1       -47.1500 -133.7760  209  112    27819        1         NaN
2       -57.8599  -80.3684  156  243    48735        2         NaN
3       -85.0511  -96.0033   24  205    62257        3         NaN
4       -85.0511  -95.9735   24  205    62257        4         NaN
5       -85.0494  -80.8804   24  241    64705        5         NaN
6       -66.5151  -39.3818  114  343    75289        6         NaN
7       -66.5134  -39.3750  114  343    75289        7         NaN
8       -82.2142   11.2500   38  466    89528        8         NaN
9       -76.3099   22.4995   66  494    91384        9         NaN
10      -54.9371  -67.6341  171  274   126433       10         NaN
11      -54.9356  -67.6150  171  274   126433       11         NaN
12      -54.8396  -68.3124  171  272   126447       12         NaN
13      -54.8395  -68.3121  171  272   126447       13        

In [418]:
from sklearn import linear_model
from sklearn import neural_network
from sklearn import ensemble
import ipdb
import time
stages = [1,10,100]
index = []
data = []

start_time=time.time()

for i in range(len(stages)):
    temp = []
    for j in range(stages[i]):
        df_ = pd.DataFrame(columns=COL)
        temp.append(df_)
    data.append(temp)
    
data[0][0] = df
nodatacount = 0

for i in range(len(stages)):
    temp = []
    print("======stage:"+str(i)+"======")
    
    for j in range(stages[i]):
        print("model:"+str(j))
        if len(data[i][j]) == 0:
            nodatacount += 1
            data[i][j] = data[i][j-1]
            print(j)
#         lr = linear_model.LinearRegression()
#         lr = linear_model.ElasticNet(max_iter=10000)
#         lr = linear_model.Lasso(max_iter=10000)
        lr = ensemble.GradientBoostingRegressor()
#         lr = ensemble.AdaBoostRegressor()
#         if i == 0:
# #             lr = neural_network.MLPRegressor(hidden_layer_sizes=(8, 2),max_iter=1000)
#             lr = ensemble.GradientBoostingRegressor(n_estimators=500)
#         else:
#             lr = ensemble.GradientBoostingRegressor()
        train_x = data[i][j].iloc[:,4].reshape(-1,1)
#         train_x = data[i][j].iloc[:,0:5]
        train_y = data[i][j].iloc[:,5]
        lr.fit(train_x,train_y)
        temp.append(lr)
        # allocate data
        if i < len(stages)-1:
            predicted = lr.predict(train_x)
            for pi in range(len(predicted)): # do not change the prediction
                #print(item)
                if predicted[pi] < 0:
                    predicted[pi] = 0
                if predicted[pi] >= TOTAL:
                    predicted[pi] = TOTAL-1   
            ModelID = ((predicted/TOTAL)*stages[i+1]).astype(int)
            data[i][j].iloc[:,-1]=ModelID
#             if i == 1 and np.any(data[i][j]['prediction'] == 4):
#                 print(j)

            for k in range(stages[i+1]):
                data[i+1][k] = data[i+1][k].append(data[i][j].loc[data[i][j]['prediction'] == k])
    index.append(temp)

elapsed_time = time.time() - start_time
print(elapsed_time)
print(nodatacount)

======stage:0======
model:0


D:\anaconda\lib\site-packages\ipykernel_launcher.py:42: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


======stage:1======
model:0
model:1
model:2
model:3
model:4
model:5
model:6
model:7
model:8
model:9
======stage:2======
model:0
model:1
model:2
model:3
model:4
model:5
model:6
model:7
model:8
model:9
model:10
model:11
model:12
model:13
model:14
model:15
model:16
model:17
model:18
model:19
model:20
model:21
model:22
model:23
model:24
model:25
model:26
model:27
model:28
model:29
model:30
model:31
model:32
model:33
model:34
model:35
model:36
model:37
model:38
model:39
model:40
model:41
model:42
model:43
model:44
model:45
model:46
model:47
model:48
model:49
model:50
model:51
model:52
model:53
model:54
model:55
model:56
model:57
model:58
model:59
model:60
model:61
model:62
model:63
model:64
model:65
model:66
model:67
model:68
model:69
model:70
model:71
model:72
model:73
model:74
model:75
model:76
model:77
model:78
model:79
79
model:80
model:81
model:82
model:83
model:84
model:85
model:86
model:87
model:88
model:89
89
model:90
model:91
model:92
model:93
model:94
model:95
model:96
model:97
mo

In [419]:
import numpy as np
# use the generated stage index for prediction
def predict_single_item_with_stage_model(model, data, total_size):
    tempy = 0
    modelIndex = 0
    for m in range(len(model)):
        tempy = model[m][modelIndex].predict(data)
#         tempy = model[m][modelIndex].predict(data.iloc[i,:].reshape(1,-1))
#         tempy = model[m][modelIndex].predict(data.iloc[i,])
        if tempy < 0:
            tempy = 0
        if tempy >= total_size:
            tempy = total_size-1
        if m < len(model)-1:
            modelIndex = int((tempy/total_size)*len(model[m+1]))
    return tempy

In [420]:
def find_discrepancy(lower_hilbert_value, upper_hilbert_value, lower_predicted_position, df):
    
    # return discrepancy, range cost, number of results, true lower and upper position
    
    predicted_lower_position_hilbert_value = df.iloc[lower_predicted_position,4]
#     if predicted_lower_position_hilbert_value - lower_hilbert_value == 0:
#         return 0
    
    shift = 0
    discrepancy = 0
    range_cost = 0
    true_lower_position = 0
    true_upper_position = 0
    number_of_results = 0

    # predicted lower position greater than the true lower position, search backward
    if predicted_lower_position_hilbert_value >= lower_hilbert_value:
        
        # the predicted lower position is even higher than the true upper position, will incur additional range search cost
        if predicted_lower_position_hilbert_value > upper_hilbert_value:
            while predicted_lower_position_hilbert_value > upper_hilbert_value:
                shift +=1
                discrepancy += 1
                range_cost += 1
                predicted_lower_position_hilbert_value = df.iloc[lower_predicted_position-shift,4]
            true_upper_position = lower_predicted_position - shift
            
            # retrieve records in this interval backwards, add range_cost
            shift = 0
            current_hilbert = df.iloc[true_upper_position, 4]
            while current_hilbert >= lower_hilbert_value:                                                                                                                                                                                                                                                                                               
                shift += 1
                discrepancy += 1
                range_cost += 1
                current_hilbert = df.iloc[true_upper_position-shift, 4]
            true_lower_position = true_upper_position-shift+1
            if true_lower_position > true_upper_position:
                number_of_results = 0
            else:
                number_of_results = true_upper_position - true_lower_position + 1
            return discrepancy, range_cost, number_of_results, true_lower_position, true_upper_position
            
        else: # in the middle of the interval
            middle_position = lower_predicted_position
            while predicted_lower_position_hilbert_value >= lower_hilbert_value:
                shift += 1
                discrepancy += 1
                range_cost += 1
                predicted_lower_position_hilbert_value = df.iloc[lower_predicted_position-shift,4]
            true_lower_position = lower_predicted_position-shift+1
            
            shift = 0
            current_hilbert = df.iloc[middle_position, 4]
            while current_hilbert <= upper_hilbert_value:
                shift += 1
                range_cost += 1
                current_hilbert = df.iloc[middle_position+shift, 4]
            true_upper_position = middle_position + shift - 1
            if true_upper_position < true_lower_position:
                number_of_results = 0
            else:
                number_of_results = true_upper_position - true_lower_position + 1
        
            # notice the lower position should be larger or equal than the lower position
            return discrepancy, range_cost, number_of_results, true_lower_position, true_upper_position
    
    else: # predicted lower position lower than the true lower position, search forward, incur addition range cost
        while predicted_lower_position_hilbert_value < lower_hilbert_value:
            shift += 1
            discrepancy += 1
            range_cost += 1
            predicted_lower_position_hilbert_value = df.iloc[lower_predicted_position+shift,4]
        
        true_lower_position = lower_predicted_position+shift
        
        # retrieve records in this interval forwards, add range_cost
        shift = 0
        current_hilbert = df.iloc[true_lower_position, 4]
        while current_hilbert <= upper_hilbert_value:
            shift += 1
            range_cost += 1
            current_hilbert = df.iloc[true_lower_position+shift, 4]
        true_upper_position = true_lower_position + shift - 1
        if true_upper_position < true_lower_position:
            number_of_results = 0
        else:
            number_of_results = true_upper_position - true_lower_position + 1
        return discrepancy, range_cost, number_of_results, true_lower_position, true_upper_position

In [421]:
# ===== Testing =====
# import ipdb
# ipdb.set_trace()
# cost1 = find_discrepancy(-2.103575, df, 0)
# print(cost1)

In [422]:
def exact_search(lower_hilbert, upper_hilbert):
    return len(df[(df['hilbert'] >= lower_hilbert) & (df['hilbert'] <= upper_hilbert)])

In [423]:
def exact_search2(lower_hilbert, upper_hilbert):
    return df[(df['hilbert'] >= lower_hilbert) & (df['hilbert'] <= upper_hilbert)]

In [424]:
def range_query(lower_x, upper_x, lower_y, upper_y,max_bits = 22, print_intervals = False, print_cost = False, model = None, df = None, scaler = None):
    lx = int((2*lower_x+180)*10000) >> 22-max_bits
    ly = int((lower_y+180)*10000) >> 22-max_bits
    ux = int((2*upper_x+180)*10000) >> 22-max_bits
    uy = int((upper_y+180)*10000) >> 22-max_bits
    
#     intervals = get_intervals(lower_x,upper_x,lower_y,upper_y,max_bits)
    intervals = get_intervals(lx,ux,ly,uy,max_bits)
    total_size = len(intervals)
    print("total intervals: ",total_size)
    if print_intervals:
        print(intervals)
    if print_cost:
        total_discrepancy = 0
        total_range_cost = 0
        total_number_of_results = 0
        total_true_count = 0
        i = 0
        frames = []
        for interval in intervals:
#             df1 = pd.DataFrame(data=[[0,0,interval["lower_x"],interval["lower_y"],interval["lower_hilbert"]]], columns=["x","y","x_","y_","hilbert"],index=[0], dtype=object)
#             df1.iloc[0,:] = scaler.transform(df1.iloc[0,:].reshape(1,-1))
#             predicted_position1 = int(predict_single_item_with_stage_model(index, df1.iloc[0,4], len(df))[0])
            predicted_position1 = int(predict_single_item_with_stage_model(index, interval["lower_hilbert"], len(df))[0])
            
#             df2 = pd.DataFrame(data=[[0,0,interval["upper_x"],interval["upper_y"],interval["upper_hilbert"]]], columns=["x","y","x_","y_","hilbert"],index=[0], dtype=object)
#             df2.iloc[0,:] = scaler.transform(df2.iloc[0,:].reshape(1,-1))
#             predicted_position2 = int(predict_single_item_with_stage_model(index, df2.iloc[0,4], len(df))[0])
            
            total_true_count += exact_search(interval["lower_hilbert"], interval["upper_hilbert"])
#             total_true_count += exact_search(df1.iloc[0,4], df2.iloc[0,4])
#             frames.append(exact_search2(interval["lower_hilbert"], interval["upper_hilbert"]))
            discrepancy, range_cost, number_of_results, true_lower_position, true_upper_position = find_discrepancy(interval["lower_hilbert"], interval["upper_hilbert"], predicted_position1, df)
#             discrepancy, range_cost, number_of_results, true_lower_position, true_upper_position = find_discrepancy(df1.iloc[0,4], df2.iloc[0,4], predicted_position1, df)
#             

            total_discrepancy += discrepancy
            total_range_cost += range_cost
            total_number_of_results += number_of_results
            
        print("total discrepancy: ",total_discrepancy)
        print("total range cost: ",total_range_cost)
        print("total results: ",total_number_of_results)
        print("true total results: ",total_true_count)
#         result = pd.concat(frames)
#         df_true = df[(df['x'] >= 22.2201) & (df['x'] <= 22.2841) & (df['y'] >= 114.1278) & (df['y'] <= 114.2572)]
#         common = df_true.merge(result,on=['order'])
#         print(common)
#         print(result)

In [426]:
# import ipdb
# ipdb.set_trace()
# range_query(1, 2, 1, 2,14)
# range_query(1, 10, 1, 10,14)
# range_query(-10, 10, -10, 10,14)
# range_query(-20, 20, -20, 20,14)
# range_query(-30, 30, -30, 30,14)
# range_query(-40, 40, -40, 40,14)
# range_query(-15.1234,17.1983,-13.1677,11.8765,14)

# range_query(38.7900,39.0118,-77.2123,-76.8835,14) # Washington DC
# range_query(22.1481,22.5457,113.8436,114.4379,14) # Hong Kong
# range_query(22.2201,22.2841,114.1278,114.2572,14) # Hong Kong island
# import ipdb
# ipdb.set_trace()
# range_query(38.0000,38.0010,-77.2123,-77.2113,22,False,True,index,df,scaler) # For Test

# range_query(38.7900,39.0118,-77.2123,-76.8835,22,False,True,index,df,scaler) # Washington DC
# range_query(22.1481,22.5457,113.8436,114.4379,22,False,True,index,df,scaler) # Hong Kong
# range_query(22.2201,22.2841,114.1278,114.2572,22,False,True,index,df,scaler) # Hong Kong island

# range_query(38.7900,39.0118,-77.2123,-76.8835,14,False,True,index,df,scaler) # Washington DC
# range_query(22.1481,22.5457,113.8436,114.4379,14,False,True,index,df,scaler) # Hong Kong
# range_query(22.2201,22.2841,114.1278,114.2572,14,False,True,index,df,scaler) # Hong Kong island

range_query(38.7900,39.0118,-77.2123,-76.8835,10,False,True,index,df,scaler) # Washington DC
range_query(22.1481,22.5457,113.8436,114.4379,10,False,True,index,df,scaler) # Hong Kong
range_query(22.2201,22.2841,114.1278,114.2572,10,False,True,index,df,scaler) # Hong Kong island

total intervals:  1
total discrepancy:  1814
total range cost:  5090
total results:  5089
true total results:  5089
total intervals:  4
total discrepancy:  173
total range cost:  1592
total results:  1574
true total results:  1574
total intervals:  2
total discrepancy:  613
total range cost:  1209
total results:  1199
true total results:  1199


In [248]:
# lx = 38.7900
# ux = 39.0118
# ly = -77.2123
# uy = -76.8835

# lx_ = int((2*38.7900+180)*10000)
# ly_ = int((-77.2123+180)*10000)
# ux_ = int((2*39.0118+180)*10000)
# uy_ = int((-76.8835+180)*10000)

# df1 = pd.DataFrame(data=[[lx,ly,lx_,ly_,0]], columns=["x","y","x_","y_","hilbert"],index=[0], dtype=object)
# df2 = pd.DataFrame(data=[[ux,uy,ux_,uy_,0]], columns=["x","y","x_","y_","hilbert"],index=[0], dtype=object)
# df1.iloc[0,:] = scaler.transform(df1.iloc[0,:].reshape(1,-1))
# df2.iloc[0,:] = scaler.transform(df2.iloc[0,:].reshape(1,-1))
# print(len(df[(df['x'] >= df1.iloc[0,0]) & (df['x'] <= df2.iloc[0,0]) & (df['y'] >= df1.iloc[0,1]) & (df['y'] <= df2.iloc[0,1])]))
# print(len(df[(df['x_'] >= df1.iloc[0,2]) & (df['x_'] <= df2.iloc[0,2]) & (df['y_'] >= df1.iloc[0,3]) & (df['y_'] <= df2.iloc[0,3])]))

# print(len(df[(df['x'] >= 22.1481) & (df['x'] <= 22.5457) & (df['y'] >= 113.8436) & (df['y'] <= 114.4379)]))

# range_query(22.1481,22.5457,113.8436,114.4379,22,False,True,index,df,scaler) # Hong Kong
# df[(df['x'] >= 22.1481) & (df['x'] <= 22.5457) & (df['y'] >= 113.8436) & (df['y'] <= 114.4379)]

# lx = 22.2201
# ux = 22.2841
# ly = 114.1278
# uy = 114.2572

# lx_ = int((2*22.2201+180)*10000)
# ly_ = int((114.1278+180)*10000)
# ux_ = int((2*22.2841+180)*10000)
# uy_ = int((114.2572+180)*10000)

# df1 = pd.DataFrame(data=[[lx,ly,lx_,ly_,0]], columns=["x","y","x_","y_","hilbert"],index=[0], dtype=object)
# df2 = pd.DataFrame(data=[[ux,uy,ux_,uy_,0]], columns=["x","y","x_","y_","hilbert"],index=[0], dtype=object)
# df1.iloc[0,:] = scaler.transform(df1.iloc[0,:].reshape(1,-1))
# df2.iloc[0,:] = scaler.transform(df2.iloc[0,:].reshape(1,-1))
# print(len(df[(df['x'] >= df1.iloc[0,0]) & (df['x'] <= df2.iloc[0,0]) & (df['y'] >= df1.iloc[0,1]) & (df['y'] <= df2.iloc[0,1])]))
# df[(df['x'] >= df1.iloc[0,0]) & (df['x'] <= df2.iloc[0,0]) & (df['y'] >= df1.iloc[0,1]) & (df['y'] <= df2.iloc[0,1])]
# print(len(df[(df['x_'] >= df1.iloc[0,2]) & (df['x_'] <= df2.iloc[0,2]) & (df['y_'] >= df1.iloc[0,3]) & (df['y_'] <= df2.iloc[0,3])]))
# print(len(df[(df['x'] >= 22.2201) & (df['x'] <= 22.2841) & (df['y'] >= 114.1278) & (df['y'] <= 114.2572)]))
# df[(df['x'] >= 22.2201) & (df['x'] <= 22.2841) & (df['y'] >= 114.1278) & (df['y'] <= 114.2572)]
# df[(df['x_'] >= 2244402) & (df['x_'] <= 2245682) & (df['y_'] >= 2941278) & (df['y_'] <= 2942572)]

x        y       x_       y_        hilbert   order  prediction
767945  22.5378  113.916  2250756  2939160  9201030726010  767945           6
767946  22.5380  113.917  2250760  2939170  9201030729324  767946           6
767947  22.5434  113.938  2250868  2939380  9201030774922  767947           6
767948  22.5432  113.930  2250864  2939300  9201030779706  767948           6
767949  22.5376  113.929  2250752  2939289  9201030786027  767949           6
767950  22.5387  113.944  2250774  2939440  9201030792724  767950           6
767951  22.5411  113.948  2250822  2939480  9201030799740  767951           6
767952  22.5424  113.950  2250848  2939500  9201030801488  767952           6
767953  22.5425  113.950  2250850  2939500  9201030801502  767953           6
767954  22.5421  113.949  2250842  2939490  9201030802786  767954           6
767955  22.5414  113.949  2250828  2939490  9201030803036  767955           6
767956  22.5402  113.954  2250804  2939540  9201030809120  767956           6
767957  22.5405  113.964  2250810  2939640  9201030814340  767957           6
767963  22.5402  113.970  2250804  2939700  9201031028874  767963           6
767964  22.5399  113.971  2250798  2939710  9201031028994  767964           6
767965  22.5399  113.971  2250798  2939710  9201031028994  767965           6
767966  22.5394  113.971  2250788  2939710  9201031029068  767966           6
767967  22.5405  113.969  2250810  2939690  9201031029640  767967           6
767968  22.5388  113.970  2250776  2939700  9201031032442  767968           6
767969  22.5401  113.988  2250802  2939880  9201031050052  767969           6
767970  22.5406  113.988  2250812  2939880  9201031050170  767970           6
767971  22.5397  113.985  2250794  2939850  9201031050786  767971           6
767972  22.5391  113.984  2250782  2939840  9201031051606  767972           6
767973  22.5387  113.984  2250774  2939840  9201031051670  767973           6
767974  22.5383  114.017  2250766  2940170  9201031119026  767974           6
767975  22.5398  114.019  2250796  2940190  9201031122182  767975           6
767976  22.5390  114.027  2250780  2940270  9201031134638  767976           6
767991  22.5383  114.049  2250766  2940490  9201032016024  767991           6
767992  22.5376  114.055  2250752  2940550  9201032020028  767992           6
767993  22.5395  114.055  2250790  2940550  9201032023682  767993           6
...         ...      ...      ...      ...            ...     ...         ...
769476  22.5400  114.115  2250800  2941150  9201116011772  769476           6
769477  22.5394  114.114  2250788  2941140  9201116011914  769477           6
769478  22.5420  114.113  2250840  2941130  9201116012932  769478           6
769479  22.5421  114.114  2250842  2941140  9201116013174  769479           6
769480  22.5411  114.114  2250822  2941140  9201116013452  769480           6
769481  22.5436  114.118  2250872  2941180  9201116015248  769481           6
769482  22.5417  114.107  2250834  2941069  9201116019549  769482           6
769483  22.5425  114.106  2250850  2941060  9201116019742  769483           6
769486  22.5448  114.111  2250896  2941110  9201116023958  769486           6
769487  22.5453  114.111  2250906  2941110  9201116024024  769487           6
769488  22.5445  114.116  2250890  2941160  9201116025998  769488           6
769489  22.5450  114.118  2250900  2941180  9201116026464  769489           6
769494  22.5441  114.105  2250882  2941050  9201116042594  769494           6
769506  22.5434  114.121  2250868  2941210  9201116222580  769506           6
769507  22.5404  114.121  2250808  2941210  9201116226604  769507           6
769508  22.5398  114.119  2250796  2941190  9201116227142  769508           6
769509  22.5392  114.120  2250784  2941200  9201116227328  769509           6
769510  22.5393  114.120  2250786  2941200  9201116227332  769510           6
769511  22.5393  114.121  2250786  2941210  9201116227554  769511           6
769512  22.5384  114.120  2250768  2

In [407]:
import csv
# HILBERT_N = 4194304 # 22 bits, 4 Million
# HILBERT_N = 16384 # 14 bits
HILBERT_N = 1024 # 1K
list = []
count = -1
with open("./Data/SortedPOIs.csv", newline='') as csvfile:
    spamreader = csv.reader(csvfile)
    for row in spamreader:
        if count == -1: # ignore the head info
            count += 1
            continue
        x = float(row[0])
        y = float(row[1])
#         x_ = int((2*x+180)*10000)
#         y_ = int((y+180)*10000)
        x_ = int((2*x+180)*10000)>>12
        y_ = int((y+180)*10000)>>12
        h_ = xy2d(HILBERT_N,x_, y_)
        list.append([x,y,x_,y_,h_,count])
        count += 1
        if count % 10000 == 0:
            print(count)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000


In [408]:
import csv
def write_csv_file(path, head, data):  
    try:  
        with open(path, 'w', newline='') as csv_file:  
            writer = csv.writer(csv_file, dialect='excel')  
  
            if head is not None:  
                writer.writerow(head)  
  
            for row in data:  
                writer.writerow(row)  
  
            print("Write a CSV file to path %s Successful." % path)  
    except Exception as e:  
        print("Write an CSV file to path: %s, Case: %s" % (path, e)) 

In [409]:
list.sort(key = lambda x:(x[4]))
for i in  range(len(list)):
    list[i][5] = i
write_csv_file("./Data/HilbertSortedPOIs.csv",['x','y','x_','y_','hilbert','order','prediction'],list)

Write a CSV file to path ./Data/HilbertSortedPOIs.csv Successful.
